Этот модуль берёт "сырой" датасет и превращает его в набор обрезанных и аугментированных картинок размерами 20 на 20 пикселей.

Работает с Датасетом российских знаков

ссылка на скачивание

https://yadi.sk/d/TX5k2hkEm9wqZ

Нужен файл на 17 Гб(!) и full-gt.csv

In [1]:
import cv2
import sklearn
import numpy as np
import os
import fnmatch
import csv
import tqdm

выгрузка картинок из
D:\\ComputerVision\\_robocar\\laneDetection\\_scraps\\06_SVM_RoadSignDetector\\full-frames\\rtsd-frames
full-gt.csv

In [2]:
rus_filepath = 'D:\\ComputerVision\\_robocar\\laneDetection\\_scraps\\06_SVM_RoadSignDetector\\full-frames\\rtsd-frames'
rus_csvpath = 'D:\\ComputerVision\\_robocar\\laneDetection\\_scraps\\06_SVM_RoadSignDetector\\full-frames\\full-gt.csv'

rus_filelist = os.listdir(rus_filepath)
print('number of rus images:{}'.format(len(rus_filelist)))

number of rus images:179138


Метод, выставляющий правильное значение класса

In [4]:
def newLabelsGen_rus(signClass):
    label = {
        '4_1_1': 4,
        '4_1_2': 3,
        '4_1_3': 2,
        '4_1_4': 5,
        '4_1_5': 6,
        '3_1': 1
    }
    return label.get(signClass, 0)


Создание нужных папок

In [5]:
targetPath = 'D:\\ComputerVision\\_robocar\\laneDetection\\_scraps\\06_SVM_RoadSignDetector\\ImgTrainRus\\'
targetFolders = ['','0\\','1\\','2\\','3\\','4\\','5\\','6\\']

for folderName in targetFolders:
    if not os.path.exists(targetPath+folderName):
        os.makedirs(targetPath+folderName)

1.Читаем CSV

2.Находим картинку

3.Вырезаем и аугментируем

4.Преобразуем класс

5.Сохраняем в нужную папочку

In [7]:
gt_file = open(rus_csvpath)
gt_reader = csv.reader(gt_file, delimiter=';')
    
next(gt_reader) 
    
for rowID,row in enumerate(gt_reader):
    name = row[0]
    roiX1 = np.int(row[1])
    roiY1 = np.int(row[2])
    roiW = np.int(row[3])
    roiH = np.int(row[4])
    sign_class = row[5]
    signID = np.int(row[6])
    
    image = cv2.imread(rus_filepath + '\\' + name)
    h,w,_ = image.shape
    classID = newLabelsGen_rus(sign_class)
    
    filePathHead = targetPath + str(classID) + '\\' + str(rowID)
    filePathTail = '.png'
    
    image = cv2.imread(rus_filepath + '\\' + name)
    h,w,_ = image.shape
        
    #cutting image according to Roi.X1; Roi.Y1; W; H;
    cut = image[roiY1:roiY1+roiH,roiX1:roiX1+roiW, :]
    cut = cv2.resize(cut,(20, 20), interpolation = cv2.INTER_CUBIC)
    cv2.imwrite(filePathHead + filePathTail,cut)
    
    #cut augmentation
    #+1
    if (roiY1+roiH+1 < h and roiX1+roiW+1 < w):
        cut = image[roiY1+1:roiY1+roiH+1,roiX1+1:roiX1+roiW+1, :]
        cut = cv2.resize(cut,(20, 20), interpolation = cv2.INTER_CUBIC)
        cv2.imwrite(filePathHead + '_1' + filePathTail,cut)
    #-1
    if (roiY1-1 > 0 and roiX1-1 > 0):
        cut = image[roiY1-1:roiY1+roiH-1,roiX1-1:roiX1+roiW-1, :]
        cut = cv2.resize(cut,(20, 20), interpolation = cv2.INTER_CUBIC)
        cv2.imwrite(filePathHead + '_2' + filePathTail,cut)

    #+2
    if (roiY1+roiH+2 < h and roiX1+roiW+2 < w):
        cut = image[roiY1+2:roiY1+roiH+2,roiX1+2:roiX1+roiW+2, :]
        cut = cv2.resize(cut,(20, 20), interpolation = cv2.INTER_CUBIC)
        cv2.imwrite(filePathHead + '_3' + filePathTail,cut)

    #-2
    if (roiY1-2 > 0 and roiX1-2 > 0):
        cut = image[roiY1-2:roiY1+roiH-2,roiX1-2:roiX1+roiW-2, :]
        cut = cv2.resize(cut,(20, 20), interpolation = cv2.INTER_CUBIC)
        cv2.imwrite(filePathHead + '_4' + filePathTail,cut)
        
    #rotation augmentation
    #возможно в будущем...
        
gt_file.close()

In [8]:
print('Done')

Done
